In [1]:
import os
import pandas as pd
import numpy as np
import win32api

In [2]:
#checks available drives. important that G: drive appears here.
drives = win32api.GetLogicalDriveStrings()
drives = drives.split('\000')[:-1]
print (drives)

['C:\\']


In [42]:
#declare full path where the files are saved
cwd = os.path.abspath('G:\\ALL\RETAIL TRADING PODS\SOUTH\Planning\FW21\sales_launch\sl_feedback') 
#declare sheet name. template must have data as the name of the sheet
my_sheet = "MAIN"


In [43]:
#checks files inside the folder
files = os.listdir(cwd) 
print(files)

['04_ITALY_01_PATRIMI_APPAREL.xlsx', '01_ITALY_01_CIPOLLA_APPAREL.xlsx', '02_ITALY_01_DENIM LION_APPAREL.xlsx', '03_ITALY_01_GAMBARDELLA_APPAREL.xlsx', '05_ITALY_01_VADALA_APPAREL.xlsx']


In [44]:
#initialize empty df
df = pd.DataFrame()
#initialize empty list to append resutls from below loop.
appended_data = []

In [45]:
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>below loop will run 1-by-1 on all files saved in folder<<<<<<<<<<<<<<<<<<<
for file in files:
    
    #define name of the file >> import to current df
    full_file = cwd + "\\" + file
    df = pd.read_excel(full_file, sheet_name=my_sheet, ignore_index=True)
    
    #>>>>>>>>>>>>Next section cleans data (unwanted rows and columns)
    #drop all columns from sections we dont care about (eg. d, e,f, etc.)
    df=df.drop(df.filter(like='d',axis=1).columns,axis=1)
    df=df.drop(df.filter(like='e',axis=1).columns,axis=1)
    df=df.drop(df.filter(like='f',axis=1).columns,axis=1)
    df=df.drop(df.filter(like='g',axis=1).columns,axis=1)
    df=df.drop(df.filter(like='h',axis=1).columns,axis=1)
    df=df.drop(df.filter(like='x',axis=1).columns,axis=1)
     
    #renames first column called 'a' to pc9.
    df.rename(columns={'a': 'pc9'}, inplace=True)
    #other columns a1, a2, etc. had been created. this deletes all those unwanted columns
    df = df[df.columns.drop(list(df.filter(regex='a')))]
    #this deletes of all rows where pc9 is NA.
    df = df[df['pc9'].notna()]
    
    #at this stage, the df has generic column names, eg. 0, 1, 2, etc.
    #Below renames columns of dataframe to first row (which contains titles in the excel)
    df.columns = df.iloc[1]
    #This gets rid of first row (ie. title rows from excel), so only data remains in df
    df = df[2:]
    #renames column material to pc9. just to make code easier.
    df.rename(columns={'Material code': 'pc9'}, inplace=True)
    
    
    #>>>>>>>>>>>>>>this 1 line of magic unpivots the data :). amazing.
    df = df.melt(id_vars=['pc9'],var_name='key', value_name='selected')
    
    
    #>>>>> Next section creates mini dataframe 'new', to split key into separate columns (shipto and measure)
    # new data frame with split value columns 
    new = df["key"].str.split("|", n = 1, expand = True) 
    # making separate first name column from new data frame 
    df["ship_to"]= new[0] 
    # making separate last name column from new data frame 
    df["measure"]= new[1] 
    # Dropping old Name columns 
    df.drop(columns =["key"], inplace = True) 
    
    #>>>>>>>> next section appends current df into the list. it will stack one file after another.
    appended_data.append(df)
    
    #>>>>>>>End of loop. It will repeat for every file in our folder.
 

In [46]:
#>>>>>>>> this converts list into a data frame with the same name. list ceases to exist.
appended_data = pd.concat(appended_data)

In [47]:
appended_data

,pc9,selected,ship_to,measure
0,29507-0031,1,LS Bologna Gran Reno,OPTIONS_PROPOSAL
1,29507-0280,1,LS Bologna Gran Reno,OPTIONS_PROPOSAL
2,29507-0548,1,LS Bologna Gran Reno,OPTIONS_PROPOSAL
3,29507-0805,1,LS Bologna Gran Reno,OPTIONS_PROPOSAL
4,29507-0839,1,LS Bologna Gran Reno,OPTIONS_PROPOSAL
5,29507-0940,0,LS Bologna Gran Reno,OPTIONS_PROPOSAL
6,29507-0994,0,LS Bologna Gran Reno,OPTIONS_PROPOSAL
7,29507-1030,1,LS Bologna Gran Reno,OPTIONS_PROPOSAL
8,29507-1083,0,LS Bologna Gran Reno,OPTIONS_PROPOSAL
9,29507-1091,0,LS Bologna Gran Reno,OPTIONS_PROPOSAL
